<a href="https://colab.research.google.com/github/rajeevradnair/LLM_Engineering/blob/main/company_brochure_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
!pip install openai

In [67]:
pip install iPython

In [68]:
pip install ipython

In [69]:
from google.colab import userdata
from openai import OpenAI

#fetch API key to OpenAI API layer
openai_api_key=userdata.get('OPENAI_API_KEY')
openai=OpenAI(api_key=openai_api_key)

#create an OpenAI client
if openai:
  print(f'openai client created: {openai} ')

else:
  print(f'Issues in creating a client to the OpenAI API layer')

openai client created: <openai.OpenAI object at 0x7cd0b78889d0> 


In [70]:
from typing import List
from bs4 import BeautifulSoup
import requests

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

#Website parser, collects title, text, body & links on the page at a given URL
class Website:
  def __init__(self, url) -> None:
    self.url=url
    response=requests.get(url, headers=headers)
    self.body=response.content
    soup=BeautifulSoup(self.body, 'html.parser')
    self.title=soup.title.string if soup.title else 'No title found'
    if soup.body:
      for irrelevant in soup.body(["script", "style", "img", "input"]):
        irrelevant.decompose()
      self.text = soup.body.get_text(separator="\n", strip=True)
    else:
        self.text = ""
    links = [link.get('href') for link in soup.find_all('a')]

    self.links = [link for link in links if link]

  def get_contents(self):
    return f'Webpage Title:\n{self.title}\nWebPage Contents:\n{self.text}'


ws = Website("https://apple.com")
ws.links

['/',
 '/us/shop/goto/store',
 '/mac/',
 '/ipad/',
 '/iphone/',
 '/watch/',
 '/apple-vision-pro/',
 '/airpods/',
 '/tv-home/',
 '/entertainment/',
 '/us/shop/goto/buy_accessories',
 'https://support.apple.com/?cid=gn-ols-home-hp-tab',
 '/us/search',
 '/us/shop/goto/bag',
 '/iphone/',
 '/iphone/',
 '/us/shop/goto/buy_iphone',
 '/apple-watch-series-10/',
 '/apple-watch-series-10/',
 '/us/shop/goto/buy_watch/apple_watch_series_10',
 '/macbook-air/',
 '/macbook-air/',
 '/us/shop/goto/buy_mac/macbook_air',
 'https://developer.apple.com/wwdc25/',
 'https://developer.apple.com/wwdc25/',
 '/shop/trade-in#iphone',
 '/shop/trade-in#iphone',
 '/ipad-air/',
 '/ipad-air/',
 '/us/shop/goto/buy_ipad/ipad_air',
 '/airpods-pro/hearing-health/',
 '/airpods-pro/hearing-health/',
 '/us/shop/goto/buy_airpods/airpods_pro_2',
 '/education/college-students/',
 '/education/college-students/',
 '/apple-card/',
 '/apple-card/',
 'https://card.apple.com/apply/application?referrer=cid%3Dapy-200-10000036&start=fals

In [71]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""
print(link_system_prompt)


You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [72]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

print(get_links_user_prompt(ws))


Here is the list of links on the website of https://apple.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/us/shop/goto/store
/mac/
/ipad/
/iphone/
/watch/
/apple-vision-pro/
/airpods/
/tv-home/
/entertainment/
/us/shop/goto/buy_accessories
https://support.apple.com/?cid=gn-ols-home-hp-tab
/us/search
/us/shop/goto/bag
/iphone/
/iphone/
/us/shop/goto/buy_iphone
/apple-watch-series-10/
/apple-watch-series-10/
/us/shop/goto/buy_watch/apple_watch_series_10
/macbook-air/
/macbook-air/
/us/shop/goto/buy_mac/macbook_air
https://developer.apple.com/wwdc25/
https://developer.apple.com/wwdc25/
/shop/trade-in#iphone
/shop/trade-in#iphone
/ipad-air/
/ipad-air/
/us/shop/goto/buy_ipad/ipad_air
/airpods-pro/hearing-health/
/airpods-pro/hearing-health/
/us/shop/goto/buy_airpods/airpods_pro_2
/education/college-student

In [73]:
MODEL = 'gpt-4o-mini'

import json
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

get_links('https://google.com')

{'links': [{'type': 'about page',
   'url': 'https://about.google/?fg=1&utm_source=google-US&utm_medium=referral&utm_campaign=hp-header'},
  {'type': 'products page',
   'url': 'https://www.google.com/intl/en/about/products'},
  {'type': 'services page',
   'url': 'https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpbfooter&fg=1'},
  {'type': 'careers page', 'url': 'https://careers.google.com/'}]}

In [74]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

print(get_all_details("https://google.com"))


Found links: {'links': [{'type': 'about page', 'url': 'https://about.google/?fg=1&utm_source=google-US&utm_medium=referral&utm_campaign=hp-header'}, {'type': 'products page', 'url': 'https://www.google.com/intl/en/about/products'}, {'type': 'services page', 'url': 'https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpbfooter&fg=1'}, {'type': 'how search works page', 'url': 'https://google.com/search/howsearchworks/?fg=1'}, {'type': 'societal impact page', 'url': 'https://ai.google/societal-impact/?utm_source=googlehpfooter&utm_medium=housepromos&utm_campaign=bottom-footer'}]}
Landing page:
Webpage Title:
Google
WebPage Contents:
About
Store
Gmail
Images
Sign in
See more
Delete
Delete
Report inappropriate predictions
Advertising
Business
How Search works
Applying AI towards science and the environment
Privacy
Terms
Settings
Search settings
Advanced search
Your data in Search
Search history
Search help
Sen

In [75]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

get_brochure_user_prompt("Google", "https://google.com")


Found links: {'links': [{'type': 'about page', 'url': 'https://about.google/?fg=1&utm_source=google-US&utm_medium=referral&utm_campaign=hp-header'}, {'type': 'products page', 'url': 'https://www.google.com/intl/en/about/products'}, {'type': 'services page', 'url': 'https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpbfooter&fg=1'}, {'type': 'careers page', 'url': 'https://careers.google.com/'}]}


"You are looking at a company called: Google\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nGoogle\nWebPage Contents:\nAbout\nStore\nGmail\nImages\nSign in\nSee more\nDelete\nDelete\nReport inappropriate predictions\nAdvertising\nBusiness\nHow Search works\nApplying AI towards science and the environment\nPrivacy\nTerms\nSettings\nSearch settings\nAdvanced search\nYour data in Search\nSearch history\nSearch help\nSend feedback\nDark theme: Off\nGoogle apps\n\nabout page\nWebpage Title:\nAbout Google: Our products, technology and company information - About Google\nWebPage Contents:\nJump to Content\nAbout\nProducts\nCompany Info\nNews\nGoogle I/O 2025\nCatch up on everything we announced at I/O — from all the latest model updates,\n                    to new and helpful ways we’re bringing AI into our products.\nLearn more\nproducts\nExplore our products and 

In [76]:
#from ipython.display import Markdown, display, update_display
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    print(result)

create_brochure("Google", "https://google.com")


Found links: {'links': [{'type': 'about page', 'url': 'https://about.google/?fg=1&utm_source=google-US&utm_medium=referral&utm_campaign=hp-header'}, {'type': 'products page', 'url': 'https://www.google.com/intl/en/about/products'}, {'type': 'ads services page', 'url': 'https://www.google.com/intl/en_us/ads/?subid=ww-ww-et-g-awa-a-g_hpafoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpafooter&fg=1'}, {'type': 'services page', 'url': 'https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpbfooter&fg=1'}, {'type': 'how search works page', 'url': 'https://google.com/search/howsearchworks/?fg=1'}, {'type': 'societal impact page', 'url': 'https://ai.google/societal-impact/?utm_source=googlehpfooter&utm_medium=housepromos&utm_campaign=bottom-footer'}]}
# Google Brochure

---

## About Us

Welcome to Google! Established to organize the world's information, Google has evolved into a multifa